In [451]:
import csv
import requests
import xml.etree.ElementTree as ET 
import pandas as pd 

In [452]:
tree = ET.parse('ioofservicT1-Metadata.xml')

In [453]:
root = tree.getroot()

### Get all table fields info

In [490]:
fields = []

In [491]:
for index, child in enumerate(root[0][1]):
    for item in child:
        if 'EntityType' in child.tag:
            field = item.attrib
            field['Table'] = child.attrib['Name']
            fields.append(field)
        else:
            continue

In [492]:
df_fields = pd.DataFrame.from_records(fields)

In [493]:
df_fields

,Table,Name,Type,Nullable,{http://www.successfactors.com/edm/sap}required,{http://www.successfactors.com/edm/sap}creatable,{http://www.successfactors.com/edm/sap}updatable,{http://www.successfactors.com/edm/sap}upsertable,{http://www.successfactors.com/edm/sap}visible,{http://www.successfactors.com/edm/sap}sortable,...,FromRole,ToRole,{http://www.successfactors.com/edm/sap}field-control,{http://www.successfactors.com/edm/sap}picklist,{http://www.successfactors.com/edm/sap}inlineRequired,Precision,Scale,DefaultValue,{http://www.successfactors.com/edm/sap}sensitive-personal-data,{http://www.successfactors.com/edm/sap}elm-strength
0,Entity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Entity,deletable,Edm.Boolean,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Entity,deletablePath,Edm.String,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Entity,effectiveDated,Edm.Boolean,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Entity,insertable,Edm.Boolean,true,false,false,false,false,true,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13195,NominationTarget,nominationId,Edm.Int64,false,true,false,false,false,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13196,NominationTarget,nominationType,Edm.Int32,false,true,false,false,false,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13197,NominationTarget,positionNav,NaN,NaN,false,false,false,false,true,true,...,NominationTarget,Position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13198,NominationTarget,successorNav,NaN,NaN,false,false,false,false,true,true,...,NominationTarget,Successor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get all table level info

In [494]:
tables = []

In [495]:
for index, child in enumerate(root[0][0][0]):
    if 'EntitySet' in child.tag:
        table = child.attrib
        for grand_child in child:
            try:
                table['Description'] = grand_child[1].text
            except:
                table['Description'] = ''
        tables.append(table)

In [496]:
df_tables = pd.DataFrame.from_records(tables)

In [497]:
df_tables.shape

(620, 8)

### Extract relationship

In [498]:
associations = []

In [499]:
for index, child in enumerate(root[0][0][0]):
    # print(child.tag)
    for item in child:
        if 'AssociationSet' in child.tag:
            association = child.attrib
            association['ToTable'] = child[1].attrib['EntitySet']
        else:
            continue
    associations.append(association)

In [500]:
df_associations = pd.DataFrame.from_records(associations).drop_duplicates()

### Post extract processing

##### 1. df_fields table

In [501]:
df_fields['Type'] = df_fields['Type'].str.replace('Edm.', '')

In [503]:
cols = df_fields.columns.to_list()

In [504]:
for idx, col in enumerate(cols):
    if '{http://www.successfactors.com/edm/sap}' in col:
        col = col.replace('{http://www.successfactors.com/edm/sap}', '')
        cols[idx] = col
    else:
        continue

In [505]:
df_fields.columns = cols

In [507]:
df_fields = df_fields.loc[df_fields['Table']!='Entity']
df_fields = df_fields.loc[df_fields['Name'].notnull()]
df_fields.shape

(12564, 26)

In [508]:
df_fields['AssetType'] = 'Column'

In [512]:
df_fields.drop(columns='Table', inplace=True)

KeyError: "['Table'] not found in axis"

In [513]:
df_fields.head()

,Name,Type,Nullable,required,creatable,updatable,upsertable,visible,sortable,filterable,...,ToRole,field-control,picklist,inlineRequired,Precision,Scale,DefaultValue,sensitive-personal-data,elm-strength,AssetType
18,PaymentInformationDetailV3_externalCode,Int64,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Column
19,PaymentInformationV3_effectiveStartDate,DateTime,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Column
20,PaymentInformationV3_worker,String,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Column
21,accountType,String,true,false,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Column
22,createdBy,String,true,false,false,false,false,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Column


##### 2. df_tables

In [535]:
df_tables.columns

Index(['Name', 'EntityType', 'label', 'creatable', 'updatable', 'upsertable',
       'deletable', 'Description', 'AssetType'],
      dtype='object')

In [536]:
cols = df_tables.columns.to_list()

In [537]:
for idx, col in enumerate(cols):
    if '{http://www.successfactors.com/edm/sap}' in col:
        col = col.replace('{http://www.successfactors.com/edm/sap}', '')
        cols[idx] = col
    else:
        continue

In [538]:
df_tables.columns = cols

In [539]:
cols

['Name',
 'EntityType',
 'label',
 'creatable',
 'updatable',
 'upsertable',
 'deletable',
 'Description',
 'AssetType']

In [540]:
df_tables['AssetType'] = 'Table'

In [542]:
df_tables = df_tables.loc[df_tables['Name']!='Entity']

##### 3. df_associations

In [543]:
cols = df_associations.columns.to_list()

In [544]:
for idx, col in enumerate(cols):
    if '{http://www.successfactors.com/edm/sap}' in col:
        col = col.replace('{http://www.successfactors.com/edm/sap}', '')
        cols[idx] = col
    else:
        continue

In [545]:
df_associations.columns = cols

In [546]:
df_associations = df_associations[['Association', 'ToTable']]

### Merge tables

In [547]:
df_fields.tail(2)

,Name,Type,Nullable,required,creatable,updatable,upsertable,visible,sortable,filterable,...,ToRole,field-control,picklist,inlineRequired,Precision,Scale,DefaultValue,sensitive-personal-data,elm-strength,AssetType
13198,successorNav,NaN,NaN,false,false,false,false,true,true,true,...,Successor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Column
13199,talentPoolNav,NaN,NaN,false,false,false,false,true,true,true,...,TalentPool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Column


In [548]:
df_associations.head(2)

,Association,ToTable
0,SFOData.mdfSystemRecordStatusNav_of_EmpCostDis...,MDFEnumValue
620,SFOData.mdfSystemStatusNav_of_TimeAccountDetail,MDFEnumValue


In [549]:
df = pd.merge(df_fields, df_associations, left_on='Relationship', right_on='Association', how='left')

In [550]:
df.shape

(12564, 28)

In [551]:
df = pd.concat([df, df_tables])

In [552]:
df.columns

Index(['Name', 'Type', 'Nullable', 'required', 'creatable', 'updatable',
       'upsertable', 'visible', 'sortable', 'filterable', 'label',
       'CollectionKind', 'display-format', 'MaxLength', 'Relationship',
       'FromRole', 'ToRole', 'field-control', 'picklist', 'inlineRequired',
       'Precision', 'Scale', 'DefaultValue', 'sensitive-personal-data',
       'elm-strength', 'AssetType', 'Association', 'ToTable', 'EntityType',
       'deletable', 'Description'],
      dtype='object')

In [553]:
df.drop(columns=['Relationship', 'FromRole', 'ToRole', 'field-control', 'Association', 'EntityType'], inplace=True)

In [554]:
df

,Name,Type,Nullable,required,creatable,updatable,upsertable,visible,sortable,filterable,...,inlineRequired,Precision,Scale,DefaultValue,sensitive-personal-data,elm-strength,AssetType,ToTable,deletable,Description
0,PaymentInformationDetailV3_externalCode,Int64,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
1,PaymentInformationV3_effectiveStartDate,DateTime,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
2,PaymentInformationV3_worker,String,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
3,accountType,String,true,false,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
4,createdBy,String,true,false,false,false,false,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,WorkScheduleDayModelSegment,NaN,NaN,NaN,true,true,true,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,true,A WorkScheduleDayModelSegment used within a Wo...
616,EMMonitoredProcess,NaN,NaN,NaN,true,false,true,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,false,None
617,FormReviewFeedbackList,NaN,NaN,NaN,false,false,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,false,It contains review feedback list for a form
618,TimeTypeAUS,NaN,NaN,NaN,false,false,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,false,Contains the time type australia definition.


In [555]:
df.to_excel('sf_schema.xlsx', index=False)

In [556]:
df

,Name,Type,Nullable,required,creatable,updatable,upsertable,visible,sortable,filterable,...,inlineRequired,Precision,Scale,DefaultValue,sensitive-personal-data,elm-strength,AssetType,ToTable,deletable,Description
0,PaymentInformationDetailV3_externalCode,Int64,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
1,PaymentInformationV3_effectiveStartDate,DateTime,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
2,PaymentInformationV3_worker,String,false,true,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
3,accountType,String,true,false,true,true,true,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
4,createdBy,String,true,false,false,false,false,true,true,true,...,NaN,NaN,NaN,NaN,NaN,NaN,Column,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,WorkScheduleDayModelSegment,NaN,NaN,NaN,true,true,true,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,true,A WorkScheduleDayModelSegment used within a Wo...
616,EMMonitoredProcess,NaN,NaN,NaN,true,false,true,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,false,None
617,FormReviewFeedbackList,NaN,NaN,NaN,false,false,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,false,It contains review feedback list for a form
618,TimeTypeAUS,NaN,NaN,NaN,false,false,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Table,NaN,false,Contains the time type australia definition.
